### Generative Adversarial Model - Group 9 <div style="text-align: right; float: right"> IANNWTF - Sheet08 </div>
In this sheet we used a generative model on the MNIST data set to generate "fake" handwritten numbers.

The training process is saved as summaries and saved in the `summary_dir` directory (it's called summary...). In tensorboard you can find the graphs for the losses, and can look at the images over the course of training. So please execute code to access graphs.

##### The usual imports

In [1]:
import numpy as np
import struct, os, time
import tensorflow as tf
import json

C:\Users\nipsh\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


##### Lukas' helper script for loading the mnist data

In [2]:
# %load recipes.py
import pymongo, json, numpy as np
from collections import Counter
from nltk.tokenize import RegexpTokenizer


class Recipes:

  def __init__(
      self,
      useMongo=True,
      config="config.json"
    ):

    with open(config) as cfg:
      self.data = json.load(cfg)
      storage = self.data['storage']
      CRAWL_FOLDER = storage['folder']
      RECIPE_COLLECTION = storage['collection']

    client = pymongo.MongoClient('mongodb://localhost:27017')
    self.db = client['iannwtf']
    self.coll = self.db[RECIPE_COLLECTION]

  def create_dictionaries(self,vocabulary_size=20000):
    agg = self.coll.aggregate([
        {"$project": {"_id": 0, "ingredients.foodId": 1, "ingredients.name": 1}},
        {"$unwind": "$ingredients"},
        {
            "$group": {
                "_id": "$ingredients.foodId",
                "name": {"$first": "$ingredients.name"}
            }
        }
      ], allowDiskUse=True)
    
    

    self._ing2id = {item['name'].lower(): int(item['_id']) for item in agg}
    self._ing2id['miracel whipmyass'] = 1362
    self._ing2id['NO_INGREDIENT'] = 0
    self._id2ing = dict(zip(self._ing2id.values(), self._ing2id.keys()))

    self._word2id = {}
    self._id2word = {} 
    self._unit2id = {}
    self._id2unit = {}

    ingredient_size = len(self._ing2id)
    unit_size = len(self._unit2id)

    return ingredient_size, unit_size, vocabulary_size

  def ings2ids(self, ings):
    return self._2(ings, self._ing2id, 0)
  
  def units2ids(self, units):
    return self._2(units, self._unit2id, 0)

  def words2ids(self, words):
    return self._2(words, self._word2id, 0)

  def ids2ings(self, ids):
    return self._2(ids, self._id2ing, "NO_INGREDIENT")
  
  def ids2units(self, ids):
    return self._2(ids, self._id2unit, "NO_UNIT")

  def ids2words(self, ids):
    return self._2(ids, self._id2word, "UNKNOWN")

  def _2(self, names, d, undef):
    if type(names) in [list, range]:
      return [d.get(n, undef) for n in names]
    elif type(names) == np.ndarray:
      res = np.full_like(names, "", dtype=object)
      for i in range(res.shape[0]):
        res[i] = self._2(names[i], d, undef)
      return res
    else:
      return d.get(names, undef)

  def get_ingredient_batch(self, batchsize=25, max_ingredients=15):
    count = 0
    batch = np.zeros((batchsize, max_ingredients), dtype=np.float32)
    cursor = self.coll.find({}, {"ingredients.foodId": 1, "_id": 0}, no_cursor_timeout = True)

    recipes = np.array(list(cursor))
    np.random.shuffle(recipes)

    for doc in recipes:
      _ids = [int(x['foodId']) for x in doc['ingredients']]
      if len(_ids) > max_ingredients:
          _ids = _ids[:max_ingredients]
      batch[count][:len(_ids)] = _ids[:]
      count += 1

      if count == batchsize:
          yield batch
          count = 0
          batch = np.zeros((batchsize, max_ingredients), dtype=np.float32)


##### Lukas' layer helper file
The `back_conv_layer` was implemented by us. It is mostly analogue to the `conv_layer` function, with the biggest difference being the `target_shape` parameter, which holds the output shape to be genereated by the transposed convolution.

In [3]:
# %load 08_mnist-gan-layers.py
def feed_forward_layer(x, target_size, normalize = False, activation_function = None):
    print("Forward-Layer:" + str(x.shape))
    
    fan_in = int(x.shape[-1])
    
    if activation_function == tf.nn.relu:
        var_init = tf.random_normal_initializer(stddev = 2/fan_in)
    else:
        var_init = tf.random_normal_initializer(stddev = fan_in**(-1/2))
    weights = tf.get_variable("weights", [x.shape[1], target_size], tf.float32, var_init)
    
    var_init = tf.constant_initializer(0.0)
    biases = tf.get_variable("biases", [target_size], tf.float32, var_init)
    
    activation = tf.matmul(x, weights) + biases
    
    if normalize:
        activation = batch_norm(activation, [0])
    
    return activation_function(activation) if callable(activation_function) else activation


def conv_layer(x, kernel_quantity, kernel_size, stride_size, normalize = False, activation_function = False):
    print("Conv-Layer:" + str(x.shape))
    depth = x.shape[-1]
    fan_in = int(x.shape[1] * x.shape[2])
    
    if activation_function == tf.nn.relu or activation_function == tf.nn.leaky_relu:
        var_init = tf.random_normal_initializer(stddev = 2/fan_in)
    else:
        var_init = tf.random_normal_initializer(stddev = fan_in**(-1/2))
    kernels = tf.get_variable("kernels", [kernel_size, kernel_size, depth, kernel_quantity], tf.float32, var_init)
    
    var_init = tf.constant_initializer(0.0)
    biases = tf.get_variable("biases", [kernel_quantity], initializer = var_init)
    
    activation = tf.nn.conv2d(x, kernels, strides = [1, stride_size, stride_size, 1], padding = "SAME") + biases
    
    if normalize:
        activation = batch_norm(activation, [0, 1, 2])
    
    return activation_function(activation) if callable(activation_function) else activation


def back_conv_layer(x, target_shape, kernel_size, stride_size, normalize = False, activation_function = False):
    print("De-Conv-Layer:" + str(x.shape))
    depth = x.shape[-1]                   #number of activation maps in the de_conv layer (in_channels)
    fan_in = int(x.shape[1] * x.shape[2]) #resolution of the activation map, important for proper intitialization of kernel weights
    kernel_quantity = target_shape[-1]    #number of activation maps in the target layer (out_channel)
    
    if activation_function == tf.nn.relu or activation_function == tf.nn.leaky_relu:
        var_init = tf.random_normal_initializer(stddev = 2/fan_in)
    else:
        var_init = tf.random_normal_initializer(stddev = fan_in**(-1/2))
        
    # switch kernel_quantity and depth (in_channels and out_channels) because we do deconvolution
    kernels = tf.get_variable("kernels", [kernel_size, kernel_size, kernel_quantity, depth], tf.float32, var_init)
    
    var_init = tf.constant_initializer(0.0)
    biases = tf.get_variable("biases", [kernel_quantity], initializer = var_init)
    
    # use conv2d_transpose as instructed
    activation = tf.nn.conv2d_transpose(x, kernels, target_shape, strides = [1, stride_size, stride_size, 1], padding = "SAME") + biases
    
    if normalize:
        activation = batch_norm(activation, [0, 1, 2])
    
    return activation_function(activation) if callable(activation_function) else activation

def flatten(x):
    size = int(np.prod(x.shape[1:]))
    return tf.reshape(x, [-1, size])

def _pop_batch_norm(x, pop_mean, pop_var, offset, scale):
    return tf.nn.batch_normalization(x, pop_mean, pop_var, offset, scale, 1e-6)

def _batch_norm(x, pop_mean, pop_var, mean, var, offset, scale):
    decay = 0.99
    
    dependency_1 = tf.assign(pop_mean, pop_mean * decay + mean * (1 - decay))
    dependency_2 = tf.assign(pop_var, pop_var * decay + var * (1 - decay))

    with tf.control_dependencies([dependency_1, dependency_2]):
        return tf.nn.batch_normalization(x, mean, var, offset, scale, 1e-6)

def batch_norm(x, axes):
    depth = x.shape[-1]
    mean, var = tf.nn.moments(x, axes = axes)
    global is_training
    
    var_init = tf.constant_initializer(0.0)
    offset = tf.get_variable("offset", [depth], tf.float32, var_init)
    var_init = tf.constant_initializer(1.0)
    scale = tf.get_variable("scale", [depth], tf.float32, var_init)
    
    pop_mean = tf.get_variable("pop_mean", [depth], initializer = tf.zeros_initializer(), trainable = False)
    pop_var = tf.get_variable("pop_var", [depth], initializer = tf.ones_initializer(), trainable = False)
    
    return tf.cond(
        is_training,
        lambda: _batch_norm(x, pop_mean, pop_var, mean, var, offset, scale),
        lambda: _pop_batch_norm(x, pop_mean, pop_var, offset, scale)
    )


##### Loading the MNIST data and hyperparameters
`batch_size` is used for the fake and the real images

In [4]:
config = "config.json"
recipes = Recipes(config)

with open(config,"r",encoding="utf-8") as cfg:
    settings = json.load(cfg)["network"]

hyperparams = settings["hyperparameters"]
directories = settings["directories"]

epochs = 2
learning_rate = 0.0004
beta1 = 0.5
batch_size = 50

#gen_feature_maps = [64, 32, 16] # target shapes for generator layers

# the pretrained weights
embedding_weights = directories["embedding_weights"]

# for tensorboard
weight_dir = directories["weight_dir"]
summary_dir = directories["summary_dir"]
     
## Hyperparameters
embedding_size = hyperparams["embedding_size"]
lstm_memory_size = hyperparams["lstm_memory_size"]
dropout_rate = hyperparams["dropout_rate"]
optimizer = tf.train.AdamOptimizer

z_size = hyperparams["z_size"]
max_ingredients = hyperparams["max_ingredients"]

#dis_feature_maps = [ 8, 16, 32] # target shapes for discriminator layers. maybe to be changed? dunno
ingredient_size, unit_size, vocabulary_size = recipes.create_dictionaries()

In [5]:
def get_ings(embeddings, ingredients):
    '''
    embeddings: the weight matrix with all ingredients representations shape: [ingredient_size, lstm_memory_size]
    ingredients: a matrix of ingedient vectors of who we want to find the nearest neigbhours
      
    '''''
    normed_embedding = tf.nn.l2_normalize(embeddings, axis=1)
    if (type(ingredients)!= list):
        arrays = ingredients
    #else: # we shouldn't have IDs, delete later
     #   ingredients = np.array(ingredients)
      #  arrays = tf.nn.embedding_lookup(embeddings, ingredients)
    
    # initalize "recipes". not sure yet how or if the loop thing is even the optimal way to do it
    
    recipes = []
    
    arraylist = tf.unstack(arrays)
    
    for array in arraylist : # get each recipe (sample in batch_size)
        
        normed_array = tf.nn.l2_normalize(array, axis=1) # what is this for?
  
        cosine_similarity = tf.matmul(normed_array, tf.transpose(normed_embedding, [1, 0]))
        
        closest_ings = tf.argmax(cosine_similarity, 1)  # shape [max_ingredients], type int64. we only need 1 ingredient, so no need for top_k anymore
        
        # stack this on top of recipes so we get batch_size, max_ingredients as output size in the end.
        # todo: actually implement.
        recipes = recipes + [closest_ings]
    
    recipes = tf.stack(recipes)
    return recipes # returns indices of closest ingredients. let's hope the dimension stuff works out in this function

In [6]:
# Defining the graph
tf.reset_default_graph()


###### The generator network. ######      
with tf.variable_scope("generator"):
  
    # see homework for original gen network

    ## placeholders
    with tf.variable_scope("inputs"):
        input_vec = tf.placeholder(tf.float32,[batch_size,z_size],name="input_vec") # for random input
        is_training = tf.placeholder(tf.bool, [],name="is_training")
        
        # what do the dimensions of this denote again?
        # i put *max_ingredients so it doesn't crash. like where I created the BasicLSTMCell a couple of lines later. how many "units" does it need?
        cell_state = tf.placeholder(tf.float32,[batch_size, lstm_memory_size*max_ingredients],name="cell_state")
        hidden_state = tf.placeholder(tf.float32,[batch_size, lstm_memory_size*max_ingredients],name="hidden_state")
        
        # original recipe placeholder. should be batch_size, max_ingredients, 3 later but for now we'll only take ingreidnets
        #orig_recipes = batch_norm(tf.placeholder(tf.int64, [batch_size,max_ingredients]), [0,1,2])

        
    ##### rnn
    with tf.variable_scope("rnn"):
        cell = tf.nn.rnn_cell.BasicLSTMCell(lstm_memory_size*max_ingredients)
        cell= tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=dropout_rate)
        
        zero_state = cell.zero_state(batch_size, tf.float32)
        #print(tf.shape(zero_state))
        #zero_state = tf.random_normal([batch_size,]) # randomize zero_state
        
        
        state = tf.nn.rnn_cell.LSTMStateTuple(c = cell_state, h = hidden_state) # should be fine so far? no changes needed?
        
        outputs, state = tf.nn.static_rnn(cell, [input_vec], initial_state = state)
        
        # so at least in the example, lstm_memory_size == embedding_size.
        # so what do we do about the subsequence length?
        # for now I'll just ignore since we haven't even used sequences but the input_vec only. veeeery unsure about
        # this though.
        # so here I will state what the output should look like but not sure how we get the LSTM to produce this exact amount of data
        outputs = tf.reshape(tf.concat(outputs, axis=1), [batch_size, max_ingredients, lstm_memory_size])


    ##### embedding
    with tf.variable_scope("embedding"):
        init = tf.random_uniform_initializer(-1.0,1.0)
        embeddings = tf.get_variable("embed", [ingredient_size, embedding_size], initializer=init, trainable=False)

        ##fake_recipes will have dimension batch_size*max_ingredients*3 just like the original ones (for now without 3)             
        # so get_ings will have to return batch_size*max_ingredients for now.
        # might or might not work at the current state :D
        #fake_recipes = get_ings(embeddings,outputs) 
                
            
    train_gen = tf.get_variable_scope().get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)


##### The discriminator network #####
# how do I adapt this?

with tf.variable_scope("discriminator"):
    
    
    orig_recipes_placeholder = tf.placeholder(tf.int64, [batch_size,max_ingredients], name="orig_recipes_placeholder")
    #orig_recipes = batch_norm(orig_recipes_placeholder, [0,1])
    #orig_recipes = orig_recipes_placeholder
    orig_recipes = tf.nn.embedding_lookup(embeddings, orig_recipes_placeholder)

    print(orig_recipes.get_shape())
    
    readable_outputs = get_ings(embeddings, outputs)

    # create input batch for discriminator
    batch = tf.concat([outputs, orig_recipes],axis=0)
    batch = tf.reduce_mean(batch,axis=2)

    
    with tf.variable_scope("layer1"):
        l1 = feed_forward_layer(batch,64,False,None)
    with tf.variable_scope("readout"):
        #logits = feed_forward_layer(l1,1,False,activation_function=tf.nn.sigmoid)
        logits = feed_forward_layer(l1,1,False,None)
        
    # get trainable variables for discriminator
    train_dis = tf.get_variable_scope().get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

##### Evaluation and training of the network #####
with tf.variable_scope("evaluation"):
    
    # to compute cross entropy for generator. consists of only "1" because this is the discriminator label for "real"
    gen_labels = tf.ones((batch_size,1))
    
    # only the first half of the output from the discriminator concerns the pictures produced by the generator
    # so only get first half of logits (which equals the batch_size)
    gen_logits = logits[:batch_size]
    
    # for discriminator cross entropy. first half of input are fake images ("0"), second half real ones ("1")
    dis_labels = tf.concat((tf.zeros((batch_size,1)),tf.ones((batch_size,1))),axis=0)
    dis_logits = logits
        
    gen_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=gen_labels, logits=gen_logits)
    dis_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=dis_labels, logits=dis_logits)
    
    # tensorboard stuff
    tf.summary.scalar("generator_loss", tf.reduce_mean(gen_loss))
    tf.summary.scalar("discriminator_loss", tf.reduce_mean(dis_loss))
    #tf.summary.scalar("discriminator_loss", dis_loss)
    
    
    global_step = tf.get_variable("global_step",[],tf.int32,trainable=False)
    
    # initialize optimizer
    optimizer  = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1)  
    
    # define training steps with respective variable lists
    gen_step = optimizer.minimize(gen_loss, var_list=train_gen, global_step=global_step)
    dis_step = optimizer.minimize(dis_loss, var_list=train_dis, global_step=global_step)
    
    summaries = tf.summary.merge_all()

(50, 15, 64)
Forward-Layer:(100, 15)
Forward-Layer:(100, 64)


##### This is where we train our network

In [17]:
from tensorflow.python.tools import inspect_checkpoint as chkp
from tensorflow.python import pywrap_tensorflow

reader = pywrap_tensorflow.NewCheckpointReader(embed_ckpt)
var_to_shape_map = reader.get_variable_to_shape_map()
for key in sorted(var_to_shape_map):
    print("tensor_name: ", key)
    print(reader.get_tensor(key).shape)
    
print(embeddings.shape)

tensor_name:  ingredient_embedding
(3601, 64)
(3601, 64)


In [ ]:
# HAVE ONLY MADE SMALL ADJUSTMENTS. don't think there's a point if we don't have data



with tf.Session() as session:
    ## initialising directories if they do not exist
    if not os.path.exists(weight_dir):
        os.makedirs(weight_dir)
    if not os.path.exists(summary_dir):
        os.makedirs(summary_dir)
    train_saver = tf.train.Saver()
    train_writer = tf.summary.FileWriter(summary_dir, session.graph)
    
    ## Try to load trained data and initialise variables if there is none
    ckpt = tf.train.latest_checkpoint(weight_dir)
    if ckpt is None or True:
        print("No checkpoint found. Initialising variables.")
        session.run(tf.global_variables_initializer())
    else:
        train_saver.restore(session, ckpt)    
    
    embed_saver = tf.train.Saver(var_list={'ingredient_embedding': embeddings})
    if os.path.isdir(embedding_weights):
        embed_ckpt = tf.train.latest_checkpoint(embedding_weights)
        if embed_ckpt is not None:
            embed_saver.restore(session, embed_ckpt)
        else:
            raise Exception("Embedding weights not found")
    
    
    ## Train all the epochs and save the summaries for each timestep
    for epoch in range(1, epochs+1):
        
        print("Starting epoch %d..." %epoch, end='')
        
        real_data = recipes.get_ingredient_batch(batch_size, max_ingredients) # get batch of original recipes #mnist.get_batch(batch_size)
        t = time.time()
        
        counter = 0
        
        for data in real_data:
            
            _state = session.run(zero_state)
            
            ## for each batch we need a new set of random vectors
            z_vec = np.random.uniform(-1, 1, (batch_size,z_size))
            
            # feed data into placeholders
            _feed_dict = {input_vec: z_vec, 
                         orig_recipes_placeholder: data,
                         is_training: True,
                         cell_state: _state.c,
                         hidden_state: _state.h,}
            
            # let's train our network!
            _gStep, _dStep, _state, _summ, _step,\
            _fr, _or, _logits, _dis_loss = session.run([
                                                gen_step, dis_step, state,
                                                summaries, global_step,
                                                readable_outputs, orig_recipes_placeholder,
                                                logits, dis_loss
                                                ],
                                                feed_dict=_feed_dict)
            train_writer.add_summary(_summ, _step)
            
            _fr = recipes.ids2ings(_fr)
            
    
            
            if counter % 100 == 0 :
                
                #for i in range(batch_size) :
                #        print(_fr[i][np.where(_fr[i]!='NO_INGREDIENT')])

                #print(np.stack(_logits,1))
                #print(_fr)
                print(_fr)
                #print(np.stack(_dis_loss,1))
                print("------------------------------------------")
            
            counter = counter + 1
            #print(str(_fr) + str(_fr.shape))
            
        t = time.time() - t
        est = t * (epochs - epoch)
        
        ## This is just the time formatting. Move along.
        m, s = divmod(t, 60)
        e_m, e_s = divmod(est, 60)
        
        print(("Finished epoch in {0: .0f}m{1: .2f}s. "+\
              "Estimated remaining time: {2: .0f}m{3: .2f}s").format(m,s,e_m,e_s))
        
        train_saver.save(session, weight_dir, global_step=_step)

No checkpoint found. Initialising variables.
INFO:tensorflow:Restoring parameters from embedding_weights/model.ckpt
Starting epoch 1...[['zwiebel(n)' 'ei(er)' 'hefe' 'vanillezucker' 'butter' 'honig'
  'knoblauch' 'zwiebel(n)' 'wasser' 'käse' 'hefe' 'käse' 'currypulver'
  'orangeat' 'backpulver']
 ['sahne' 'kandiszucker' 'käse' 'butter' 'sahne' 'pfeffer' 'zimt'
  'ei(er)' 'käse' 'wildfond' 'mehl' 'mehl' 'petersilie' 'petersilie'
  'eiweiß']
 ['sahne' 'eiweiß' 'pfeffer' 'petersilie' 'öl' 'paprikapulver'
  'backpulver' 'mozzarella' 'champignons' 'lauch' 'sahne' 'ei(er)'
  'schlagsahne' 'tomate(n)' 'meersalz']
 ['salatgurke(n)' 'hefe' 'limette(n)' 'wasser' 'paprikaschote(n)'
  'schweineschnitzel' 'tomate(n)' 'mascarpone' 'spargel' 'ingwer' 'apfel'
  'hackfleisch' 'zucchini' 'joghurt' 'zitronensaft']
 ['mayonnaise' 'ei(er)' 'meersalz' 'apfel' 'wasser' 'sahne'
  'weizentortilla(s)' 'dill' 'ingwer' 'joghurt' 'pinienkerne'
  'haferflocken' 'ei(er)' 'paprikapulver' 'ei(er)']
 ['salz' 'natron' '

[['basilikum' 'wasser' 'ananas' 'mehl' 'schinken' 'milch' 'butter'
  'pfeffer' 'sahne' 'salz' 'ei(er)' 'salz' 'wasser' 'ei(er)' 'mehl']
 ['chilischote(n)' 'zwiebel(n)' 'speisestärke' 'essig' 'orangensaft'
  'zitrone(n)' 'mandarine(n)' 'öl' 'petersilie' 'butter' 'hackfleisch'
  'sahne' 'milch' 'sauerteig' 'knoblauch']
 ['lorbeerblätter' 'wasser' 'milch' 'zitronensaft' 'nudeln'
  'kartoffel(n)' 'natron' 'wasser' 'mascarpone' 'gemüsebrühe' 'sojasauce'
  'butter' 'ei(er)' 'tomate(n)' 'margarine']
 ['zucker' 'salz' 'margarine' 'zimt' 'butter' 'schalotte(n)'
  'kreuzkümmel' 'pfeffer' 'mehl' 'sahnesteif' 'butter' 'wasser'
  'chilischote(n)' 'mayonnaise' 'schafskäse']
 ['petersilie' 'semmelbrösel' 'ei(er)' 'salz' 'zwiebel(n)' 'salz' 'salz'
  'gewürznelke(n)' 'knoblauch' 'ei(er)' 'sahne' 'milch' 'brötchen'
  'butter' 'mehl']
 ['roggenmehl' 'tomatenmark' 'backpulver' 'sahne' 'zucker' 'sojasauce'
  'zwiebel(n)' 'butter' 'tortenguss' 'muskat' 'salz' 'hackfleisch' 'mehl'
  'milch' 'öl']
 ['wasser' 

[['knoblauch' 'petersilie' 'currypulver' 'mehl' 'zucker' 'ananas' 'wodka'
  'wasser' 'puderzucker' 'zucker' 'salz' 'zwiebel(n)' 'chilischote(n)'
  'quark' 'schmand']
 ['zimt' 'knoblauch' 'salz' 'ei(er)' 'tomatenmark' 'gewürzgurke(n)'
  'ei(er)' 'mehl' 'mehl' 'apfel' 'butter' 'backpulver' 'möhre(n)'
  'butter' 'zucker']
 ['zimt' 'ei(er)' 'salz' 'ei(er)' 'milch' 'pfeffer' 'hackfleisch' 'sahne'
  'mehl' 'sojasauce' 'butter' 'zitrone(n)' 'ingwer' 'currypulver'
  'hackfleisch']
 ['zucker' 'mehl' 'schalotte(n)' 'ei(er)' 'ei(er)' 'zitrone(n)'
  'zitronensaft' 'mehl' 'wasser' 'sahne' 'paprikapulver' 'zitronensaft'
  'rosinen' 'gurkensalat' 'zucker']
 ['ei(er)' 'joghurt' 'wasser' 'blattspinat' 'butter' 'ei(er)' 'wasser'
  'pfeffer' 'gemüsebrühe' 'tomate(n)' 'orangensaft' 'sojasauce'
  'sojasauce' 'butter' 'mehl']
 ['zucker' 'salz' 'salatgurke(n)' 'petersilie' 'oblaten' 'ei(er)'
  'knoblauch' 'knoblauch' 'käse' 'mascarpone' 'zitrone(n)'
  'hähnchenbrustfilet(s)' 'essig' 'backpulver' 'tomate(n)']

[['limette(n)' 'petersilie' 'kardamomkapsel(n)' 'gorgonzola' 'buchweizen'
  'möhre(n)' 'ananas' 'semmelbrösel' 'ei(er)' 'ei(er)' 'käse' 'mehl'
  'ei(er)' 'hefe' 'olivenöl']
 ['haferflocken' 'koriandergrün' 'tomate(n)' 'butter' 'butter' 'wasser'
  'ei(er)' 'ei(er)' 'salz' 'zitronensaft' 'joghurt' 'gurkenflüssigkeit'
  'zimt' 'knoblauch' 'essig']
 ['butterschmalz' 'salatgurke(n)' 'wasser' 'ingwer' 'salatgurke(n)'
  'ei(er)' 'salicorne' 'käse' 'sahne' 'ei(er)' 'zitrone(n)' 'salz'
  'ei(er)' 'ei(er)' 'knoblauchzehe(n)']
 ['holzspieße' 'kaffee, gekochter' 'salz' 'paniermehl' 'ei(er)'
  'paniermehl' 'mehl' 'fleischbrühe' 'blumenkohl' 'ei(er)' 'currypulver'
  'pfeffer' 'saure sahne' 'zitronensaft' 'salatgurke(n)']
 ['zitronensaft' 'lachs' 'zitrone(n)' 'ei(er)' 'tomate(n)' 'salz'
  'zitrone(n)' 'kaninchenfilet(s)' 'ei(er)' 'salz' 'möhre(n)' 'mehl'
  'ei(er)' 'mascarpone' 'natron']
 ['ziegenquark' 'kürbis(se)' 'salz' 'käse' 'ei(er)' 'lorbeerblätter'
  'zitrone(n)' 'sojasauce' 'kürbiskerne' 'was

[['zitrone(n)' 'salz' 'ei(er)' 'ananas' 'ei(er)' 'sojasauce'
  'knoblauchzehe(n)' 'zitronensaft' 'ei(er)' 'milch' 'ei(er)' 'pfeffer'
  'eisbergsalat' 'ei(er)' 'käse']
 ['salz' 'salz' 'tomate(n)' 'zitronensaft' 'butter' 'mehl' 'sojasauce'
  'spargel' 'currypulver' 'ei(er)' 'salz' 'milchreis' 'ei(er)' 'mehl'
  'knoblauch']
 ['limettensaft' 'salz' 'mehl' 'zimt' 'zucker' 'butter' 'öl' 'essig'
  'mehl' 'gelatine' 'salz' 'pfeffer' 'wasser' 'orangeat' 'salzwasser']
 ['zitrone(n)' 'salz' 'wasser' 'butter' 'mehl' 'currypulver' 'mehl'
  'salz' 'wasser' 'basilikum' 'backpulver' 'muskat' 'zimt' 'wasser'
  'zucker']
 ['butter' 'ananassaft' 'ricotta' 'zitronensaft' 'mehl' 'hackfleisch'
  'eis' 'salz' 'lauch' 'tomatenmark' 'bambussprosse(n)' 'ei(er)' 'ei(er)'
  'zitrone(n)' 'wasser']
 ['gelatine' 'knoblauch' 'salz' 'kümmel' 'essig' 'eis' 'eigelb'
  'sojasauce' 'sahne' 'gewürzgurke(n)' 'käse' 'ei(er)' 'sauerkraut'
  'zitronensaft' 'gelatine']
 ['butterschmalz' 'ananassaft' 'puddingpulver' 'honig' 'was